<a href="https://colab.research.google.com/github/girija2204/DLTopics/blob/master/Siamese%20Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
from tensorflow.keras.models import load_model
import tensorflow.keras.backend as k
from sklearn import datasets
import numpy as np
import matplotlib.pyplot as plt
import cv2
import time

In [ ]:
def plot_face_predictions(pair, prediction, index):
    rounded_pred = round(prediction, 1)
    concatenated_img = np.concatenate((pair[0], pair[1]), axis=1)
    concatenated_img = cv2.resize(concatenated_img, (448, 224), interpolation=cv2.INTER_CUBIC)
    concatenated_img = np.float32(concatenated_img)
    concatenated_img = cv2.cvtColor(concatenated_img, cv2.COLOR_GRAY2RGB)
    if rounded_pred >= 0.5:
        cv2.putText(concatenated_img, f"Positive Match",
                    (10 + math.ceil(concatenated_img.shape[0] / 2), 0 + math.ceil(concatenated_img.shape[1] / 2)-30),
                    cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
        cv2.putText(concatenated_img, f"Similarity: {str(rounded_pred)}",
                    (10 + math.ceil(concatenated_img.shape[0] / 2), 0 + math.ceil(concatenated_img.shape[1] / 2)),
                    cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
    else:
        cv2.putText(concatenated_img, f"Similarity: {str(rounded_pred)}",
                    (60 + math.ceil(concatenated_img.shape[0] / 2), 0 + math.ceil(concatenated_img.shape[1] / 2)),
                    cv2.FONT_HERSHEY_COMPLEX, 1 / 2, (255, 0, 0), 1)
    cv2.namedWindow('image1', cv2.WINDOW_GUI_NORMAL)
    cv2.imshow("image1", concatenated_img)
    if prediction >= 0.5:
        cv2.waitKey(1000)
    else:
        cv2.waitKey(500)

In [ ]:
def generate_test_image_pairs(images_dataset, labels_dataset, image):
    unique_labels = np.unique(labels_dataset)
    label_wise_indices = dict()
    for label in unique_labels:
        label_wise_indices.setdefault(label,
                                      [index for index, curr_label in enumerate(labels_dataset) if
                                       label == curr_label])
    pair_images = []
    pair_labels = []
    for label, indices_for_label in label_wise_indices.items():
        test_image = images_dataset[np.random.choice(indices_for_label)]
        pair_images.append((image, test_image))
        pair_labels.append(label)
    return np.array(pair_images), np.array(pair_labels)

In [ ]:
def get_data(dataset, count=None, same_label=False, label=None, categories=None):
    images = dataset.images
    targets = dataset.target
    if count is None:
        return images, targets

In [ ]:
def euclidean_distance(self, vectors):
    (featA, featB) = vectors
    sum_squared = k.sum(k.square(featA - featB), axis=1, keepdims=True)
    return k.sqrt(k.maximum(sum_squared, k.epsilon()))

In [ ]:
ol_faces = datasets.fetch_olivetti_faces()
ol_images, ol_targets = get_data(dataset=ol_faces)
images_dataset = ol_images
labels_dataset = ol_targets

In [ ]:
image = cv2.imread("C:\\Users\\girij\\PycharmProjects\\other "
                   "resources\\Article\\Fifteenth\\dataset\\img\\img\\my-mg.png")
test_image = cv2.imread("C:\\Users\\girij\\PycharmProjects\\other "
                        "resources\\Article\\Fifteenth\\dataset\\img\\img\\image1_20210118100646.png")

gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray_img = gray_img / 255

gray_test_img = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)
gray_test_img = gray_test_img / 255

In [ ]:
iiii = np.append(images_dataset, [gray_img], axis=0)
llll = np.append(labels_dataset, [40], axis=0)

In [ ]:
test_image_pairs, test_label_pairs = generate_test_image_pairs(iiii, llll, gray_test_img)

In [ ]:
model = load_model("C:\\Users\\girij\\PycharmProjects\\other resources\\Article\\Fifteenth\\model",
                   custom_objects={"euclidean_distance": euclidean_distance})

In [ ]:
probs = []
for index, pair in enumerate(test_image_pairs):
    pair_image1 = np.expand_dims(pair[0], axis=-1)
    pair_image1 = np.expand_dims(pair_image1, axis=0)
    pair_image2 = np.expand_dims(pair[1], axis=-1)
    pair_image2 = np.expand_dims(pair_image2, axis=0)
    prediction = model.predict([pair_image1, pair_image2])[0][0]
    probs.append(prediction)
    print(prediction)
    plot_face_predictions(pair, prediction, index)
print(probs)
